In [29]:
import argparse
import logging
from tablebench.core import TabularDataset, TabularDatasetConfig
import matplotlib.pyplot as plt

from tablebench.configs.experiment_configs import EXPERIMENT_CONFIGS
from tablebench.models.utils import get_estimator
from tablebench.models.training import train

ModuleNotFoundError: No module named 'matplotlib'

In [4]:
experiment = 'physionet'
cache_dir = '../tmp'

In [7]:
LOG_LEVEL = logging.DEBUG

logger = logging.getLogger()
logging.basicConfig(
    format='%(asctime)s %(levelname)-8s [%(filename)s:%(lineno)d] %(message)s',
    level=LOG_LEVEL,
    datefmt='%Y-%m-%d %H:%M:%S')

In [8]:
expt_config = EXPERIMENT_CONFIGS[experiment]

In [9]:
dataset_config = TabularDatasetConfig(cache_dir=cache_dir)
tabular_dataset_kwargs = expt_config.tabular_dataset_kwargs
if "name" not in tabular_dataset_kwargs:
    tabular_dataset_kwargs["name"] = experiment

dset = TabularDataset(config=dataset_config,
                      splitter=expt_config.splitter,
                      grouper=expt_config.grouper,
                      preprocessor_config=expt_config.preprocessor_config,
                      **tabular_dataset_kwargs)

/homes/gws/jpgard/tablebench/tablebench/core/discretization.py:209: FutureWarning: In version 1.3 onwards, subsample=2e5 will be used by default. Set subsample explicitly to silence this warning in the mean time. Set subsample=None to disable subsampling explicitly.
  warnings.warn(
/homes/gws/jpgard/tablebench/tablebench/core/discretization.py:304: UserWarning: Bins whose width are too small (i.e., <= 1e-8) in feature 1 are removed. Consider decreasing the number of bins.
  warnings.warn(
/homes/gws/jpgard/tablebench/tablebench/core/discretization.py:304: UserWarning: Bins whose width are too small (i.e., <= 1e-8) in feature 34 are removed. Consider decreasing the number of bins.
  warnings.warn(
/homes/gws/jpgard/tablebench/tablebench/core/discretization.py:304: UserWarning: Bins whose width are too small (i.e., <= 1e-8) in feature 35 are removed. Consider decreasing the number of bins.
  warnings.warn(
0it [00:00, ?it/s]
0it [00:00, ?it/s]


In [10]:
X_tr, y_tr, _, _ = dset.get_pandas("train")

In [12]:
from sklearn.linear_model import LogisticRegressionCV

In [13]:
est = LogisticRegressionCV(cv=3)

In [14]:
est.fit(X_tr, y_tr)

LogisticRegressionCV(cv=3)

In [25]:
preds = est.predict(X_tr)
# errs = 

In [28]:
pd.Series(preds).value_counts()

0    609596
dtype: int64

In [21]:
est.coef_.shape

(1, 40)

In [24]:
import pandas as pd
pd.DataFrame({'name': est.feature_names_in_,
              'coef': est.coef_.ravel()}).set_index('name').sort_values('coef')

,coef
name,
kbin__DBP,-0.060502
kbin__MAP,-0.033126
kbin__O2Sat,-0.028756
kbin__SBP,-0.026194
kbin__Unit1,-0.008306
kbin__Unit2,-0.008306
kbin__Magnesium,-0.006511
kbin__Calcium,-0.005474
kbin__Platelets,-0.003461


In [31]:
pd.crosstab(X_tr.kbin__ICULOS, y_tr)

SepsisLabel,0,1
kbin__ICULOS,,
0.0,108125,1590
1.0,123817,1181
2.0,120717,834
3.0,123376,862
4.0,124978,4116


In [52]:
from tablebench.core.data_source import PhysioNetDataSource
raw_data = PhysioNetDataSource(cache_dir=cache_dir).get_data()

In [56]:
import numpy as np
raw_data['ICULOS'].quantile(np.linspace(0,1,11))

0.0      1.0
0.1      5.0
0.2      9.0
0.3     13.0
0.4     17.0
0.5     21.0
0.6     26.0
0.7     31.0
0.8     38.0
0.9     47.0
1.0    336.0
Name: ICULOS, dtype: float64